Image Reconstruction

Choose any image you like. Use Random Fourier Features (RFF) and Linear Regression to learn the mapping from the image coordinates (X, Y) to the pixel colors (R, G, B). Here, (X, Y) represents the coordinates of the pixels, and (R, G, B) represents the color values at those coordinates.

1. **Load Image**: Select any image of your choice.
2. **Random Fourier Features (RFF)**: Implement RFF to map pixel coordinates to color values.
3. **Linear Regression**: Use linear regression to learn the mapping.
4. **Display Results**: Show both the original and reconstructed images.
5. **Metrics**: Calculate the Root Mean Squared Error (MSE) and Peak Signal-to-Noise Ratio (PSNR) between the original and reconstructed images.

**Key Variables**:
- X, Y: Pixel coordinates.
- R, G, B: Pixel color values.


#### Importing necessary libraries

In [6]:
import torch
print(torch.__version__)

import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F

# Remove all the warnings
import warnings
warnings.filterwarnings('ignore')

# Set env CUDA_LAUNCH_BLOCKING=1
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Retina display
%config InlineBackend.figure_format = 'retina'

try:
    from einops import rearrange
except ImportError:
    %pip install einops

    from einops import rearrange

2.3.1+cu121


In [5]:
import torch
print(torch.cuda.get_device_name(0))
print("CUDA available:", torch.cuda.is_available())

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

#### Importing image

In [ ]:
import os

# Define the directory path
directory_path = '../assets/images/'

# Check if the directory exists, and create it if it does not
if not os.path.exists(directory_path):
    os.makedirs(directory_path)

# Check if the file exists
if os.path.exists(os.path.join(directory_path, 'dog.jpg')):
    print('dog.jpg exists')
else:
    # Download the file if it does not exist
    !wget https://segment-anything.com/assets/gallery/AdobeStock_94274587_welsh_corgi_pembroke_CD.jpg -O ../assets/images/dog.jpg


In [ ]:
# Read in a image from torchvision
img = torchvision.io.read_image("../assets/images/dog.jpg")
print(img.shape)

In [ ]:
plt.imshow(rearrange(img, 'c h w -> h w c').numpy())
plt.axis('off')
plt.show()

In [ ]:
# # Read in a image from torchvision
# iitgn_img = torchvision.io.read_image("../assets/images/iitgn.jpg")
# print(iitgn_img.shape)
# plt.imshow(rearrange(iitgn_img, 'c h w -> h w c').numpy())


In [ ]:
from sklearn import preprocessing

scaler_img = preprocessing.MinMaxScaler().fit(img.reshape(-1, 1))
scaler_img

In [ ]:

img_scaled = scaler_img.transform(img.reshape(-1, 1)).reshape(img.shape)
print(img_scaled.shape)

img_scaled = torch.tensor(img_scaled)


In [ ]:
img_scaled = img_scaled.to(device)
img_scaled

#### Crop the image

In [ ]:
crop = torchvision.transforms.functional.crop(img_scaled.cpu(), 600, 800, 300, 300)
crop.shape

In [ ]:
plt.imshow(rearrange(crop, 'c h w -> h w c').cpu().numpy())
plt.axis('off')
plt.show()

In [ ]:
crop = crop.to(device)

#### Create a coordinate map

In [ ]:
def create_coordinate_map(img):
    """
    img: torch.Tensor of shape (num_channels, height, width)

    return: tuple of torch.Tensor of shape (height * width, 2) and torch.Tensor of shape (height * width, num_channels)
    """

    num_channels, height, width = img.shape
    print("Number of channels:", num_channels, "\nHeight:", height, "\nWidth:", width)
    # Create a 2D grid of (x,y) coordinates (h, w)
    # width values change faster than height values
    w_coords = torch.arange(width).repeat(height, 1)
    h_coords = torch.arange(height).repeat(width, 1).t()
    w_coords = w_coords.reshape(-1)
    h_coords = h_coords.reshape(-1)

    # Combine the x and y coordinates into a single tensor
    X = torch.stack([h_coords, w_coords], dim=1).float()

    # Move X to GPU if available
    X = X.to(device)
    print("X shape:", X.shape)
    # Reshape the image to (h * w, num_channels)
    Y = rearrange(img, 'c h w -> (h w) c').float()
    print("Y shape:", Y.shape)

    return X, Y

In [ ]:
dog_X, dog_Y = create_coordinate_map(crop)

print(dog_X) # (300*300, 2)- coordinates
print(dog_Y) # (300*300, 3)- RGB values

In [ ]:
# MinMaxScaler from -1 to 1
scaler_X = preprocessing.MinMaxScaler(feature_range=(-1, 1)).fit(dog_X.cpu())

# Scale the X coordinates
dog_X_scaled = scaler_X.transform(dog_X.cpu())

# Move the scaled X coordinates to the GPU
dog_X_scaled = torch.tensor(dog_X_scaled).to(device)

# Set to dtype float32
dog_X_scaled = dog_X_scaled.float()

#### Reconstructing using Linear Model

In [ ]:
torch.manual_seed(42) # Set seed for reproducibility

class LinearModel(nn.Module):
    def __init__(self, in_features, out_features):
        super(LinearModel, self).__init__()
        self.linear = nn.Linear(in_features, out_features)

    def forward(self, x):
        return self.linear(x)


In [ ]:
net = LinearModel(2, 3)
net.to(device)
print(net)


In [ ]:
print("Weights:", net.linear.weight)
print("Bias:", net.linear.bias)

In [ ]:
def train(net, lr, X, Y, epochs, verbose=True):
    """
    net: torch.nn.Module
    lr: float
    X: torch.Tensor of shape (num_samples, 2)
    Y: torch.Tensor of shape (num_samples, 3)
    """
    losses = []
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)
    for epoch in range(1,epochs+1):
        optimizer.zero_grad()
        outputs = net(X)
        loss = criterion(outputs, Y)
        losses.append(loss.item())
        loss.backward()
        optimizer.step()
        if verbose and epoch % 100 == 0:
            print(f"Epoch {epoch} loss: {loss.item():.6f}")
    return loss.item(),losses

In [ ]:
train_loss,training_losses = train(net, 0.01, dog_X_scaled, dog_Y, 1000)

In [ ]:
from latexify import latexify , format_axes
import matplotlib.pyplot as plt


In [ ]:
# plot training loss graph
plt.figure(figsize=(10, 5))
latexify()
format_axes(plt.gca())
plt.plot(training_losses)
plt.title("Training Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.show()



In [ ]:
def plot_reconstructed_and_original_image(original_img, net, X, title=""):
    """
    net: torch.nn.Module
    X: torch.Tensor of shape (num_samples, 2)
    Y: torch.Tensor of shape (num_samples, 3)
    """
    num_channels, height, width = original_img.shape
    net.eval()
    with torch.no_grad():
        outputs = net(X)
        outputs = outputs.reshape(height, width, num_channels)
        #outputs = outputs.permute(1, 2, 0)
    fig = plt.figure(figsize=(6, 4))
    gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1])

    ax0 = plt.subplot(gs[0])
    ax1 = plt.subplot(gs[1])

    ax0.imshow(outputs.cpu())
    ax0.set_title("Reconstructed Image")


    ax1.imshow(original_img.cpu().permute(1, 2, 0))
    ax1.set_title("Original Image")

    for a in [ax0, ax1]:
        a.axis("off")


    fig.suptitle(title, y=0.9)
    plt.tight_layout()

In [ ]:
plot_reconstructed_and_original_image(crop, net, dog_X_scaled, title="Reconstructed Image")

#### Using Polynomial Basis Functions

In [ ]:
# Use polynomial features of degree "d"

def poly_features(X, degree):
    """
    X: torch.Tensor of shape (num_samples, 2)
    degree: int

    return: torch.Tensor of shape (num_samples, degree * (degree + 1) / 2)
    """
    X1 = X[:, 0]
    X2 = X[:, 1]
    X1 = X1.unsqueeze(1)
    X2 = X2.unsqueeze(1)
    X = torch.cat([X1, X2], dim=1)
    poly = preprocessing.PolynomialFeatures(degree=degree)
    X = poly.fit_transform(X.cpu())
    return torch.tensor(X, dtype=torch.float32).to(device)

In [ ]:
def normalize_image(image):
    """Normalize image to [0, 1] range for float images."""
    image_min = image.min()
    image_max = image.max()
    return (image - image_min) / (image_max - image_min)

def sigmoid(x):
    return 1 / (1 + torch.exp(-x))

def apply_sigmoid(image_tensor):
    """Apply sigmoid to image tensor to normalize it to [0, 1] range."""
    return sigmoid(image_tensor)

def clip(image_tensor):
    """Clip image tensor to [0, 1] range."""
    return torch.clamp(image_tensor, 0, 1)



In [ ]:
# Define polynomial degrees to test
degrees = [5, 10, 50, 100]

# Initialize lists to store training losses and images for each degree
training_losses = []
original_images = []
reconstructed_clipped_images = []

for degree in degrees:
    # Create polynomial features
    dog_X_scaled_poly = poly_features(dog_X_scaled, degree)
    print(f"Degree {degree}: {dog_X_scaled_poly.dtype}, {dog_X_scaled_poly.shape}, {dog_Y.shape}, {dog_Y.dtype}")

    # Initialize and train the model
    net = LinearModel(dog_X_scaled_poly.shape[1], 3)
    net.to(device)

    # Train the model
    train_poly_loss, losses = train(net, 0.005, dog_X_scaled_poly, dog_Y, 1500)
    training_losses.append(losses)

    # Generate reconstructed image
    with torch.no_grad():
        output = net(dog_X_scaled_poly)

    # Reshape output and apply transformations
    reconstructed_image = output.cpu().reshape(crop.shape[1], crop.shape[2], -1)
    clipped_image = clip(reconstructed_image)

    # Append images to lists with correct shape
    original_images.append(rearrange(crop, 'c h w -> h w c').cpu().numpy())
    reconstructed_clipped_images.append(clipped_image.numpy())

In [ ]:
print(original_images[0].shape)
print(reconstructed_clipped_images[0].shape)

In [ ]:
print(reconstructed_clipped_images[0][0,:5,:])

In [ ]:
# Create a figure with subplots for loss and images
fig, axs = plt.subplots(len(degrees),3, figsize=(10, len(degrees) * 3))

latexify()


for i, degree in enumerate(degrees):
    # Plot training loss
    axs[i, 0].plot(training_losses[i])
    axs[i, 0].set_title(f"Training Loss for Degree {degree}")
    axs[i, 0].set_xlabel("Epoch")
    axs[i, 0].set_ylabel("Loss")
    format_axes(axs[i, 0])

    # Plot original image
    axs[i, 1].imshow(original_images[0])
    axs[i, 1].set_title(f"Original Image")
    axs[i, 1].axis('off')

    # Plot reconstructed clipped image
    axs[i, 2].imshow(reconstructed_clipped_images[i])
    axs[i, 2].set_title(f"Reconstructed Clipped Image (Degree {degree})")
    axs[i, 2].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Plot all training losses in a single plot
plt.figure(figsize=(12, 6))
for i, degree in enumerate(degrees):
    plt.plot(training_losses[i], label=f'Degree {degree}')
plt.title("Training Loss for Different Polynomial Degrees")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()

#### Reconstructing using Random Fourier Features

In [ ]:
# create RFF features
def create_rff_features(X, num_features, sigma):
    from sklearn.kernel_approximation import RBFSampler
    rff = RBFSampler(n_components=num_features, gamma=1/(2 * sigma**2))
    X = X.cpu().numpy()
    X = rff.fit_transform(X)
    return torch.tensor(X, dtype=torch.float32).to(device)


In [ ]:
n_features = 37500
sigma = 0.008
X_rff = create_rff_features(dog_X_scaled, n_features, sigma)
print(X_rff.shape)

In [ ]:
net = LinearModel(X_rff.shape[1], 3)
net.to(device)

train_rff_loss , train_rff_losses = train(net, 0.005, X_rff, dog_Y, 2500)

In [ ]:
# plot training loss graph
plt.figure(figsize=(10, 5))
latexify()
format_axes(plt.gca())
plt.plot(train_rff_losses)
plt.title("Training Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.show()



In [ ]:
plot_reconstructed_and_original_image(crop, net, X_rff, title="Reconstructed Image with RFF Features")